In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import igraph as ig

from networkx.algorithms import community

In [ ]:
df = pd.read_csv('docs/static/data/graph.csv')
df = df[['MUNIC_MOV','MUNIC_RES','HOSPITALIZACOES']]
df = df[df['MUNIC_MOV'] != df['MUNIC_RES']]
df.head()

In [ ]:
G = nx.from_pandas_edgelist(df, 
                           source='MUNIC_MOV', 
                           target='MUNIC_RES', 
                           edge_attr='HOSPITALIZACOES',
                           create_using=nx.DiGraph())

In [ ]:
# Convert to igraph format
edges = [(df.iloc[i]['MUNIC_MOV'], df.iloc[i]['MUNIC_RES']) for i in range(len(df))]
weights = df['HOSPITALIZACOES'].tolist()
g_ig = ig.Graph.TupleList(edges, directed=True)
g_ig.es['weight'] = weights

# Apply Leiden algorithm
communities = g_ig.community_infomap(edge_weights='weight', vertex_weights=None)

print(f"Number of communities found: {len(communities)}")

In [ ]:
# Get community membership for each vertex
membership = communities.membership

# Create a DataFrame with municipality-community mapping
community_df = pd.DataFrame({
    'municipality': [g_ig.vs[i]['name'] for i in range(len(g_ig.vs))],
    'community_id': membership
})

In [ ]:
community_df.to_csv('docs/static/data/communities.csv')

In [ ]:
df = df.join(community_df.set_index('municipality'), on = 'MUNIC_RES')
df.rename(columns = { 'community_id': 'COMMUNITY_RES' }, inplace = True)
df = df.join(community_df.set_index('municipality'), on = 'MUNIC_MOV')
df.rename(columns = { 'community_id': 'COMMUNITY_MOV' }, inplace = True)


In [ ]:
community_graph = df.groupby(['COMMUNITY_RES', 'COMMUNITY_MOV']).agg({'HOSPITALIZACOES':'sum'}).reset_index()
community_graph

In [ ]:
groups = community_graph.groupby('COMMUNITY_RES')

percentages = { res: group[group['COMMUNITY_MOV'] == res]['HOSPITALIZACOES'].sum() / group['HOSPITALIZACOES'].sum() for res, group in groups}

In [ ]:
zeros = [community for community in percentages if percentages[community] == 0.0]

In [ ]:
df[df['COMMUNITY_MOV'].isin(zeros)]